In [3]:
import tensorflow as tf

In [5]:
class AlexNet(tf.keras.Model):
    def __init__ (self, num_classes=10):
        super().__init__()
        self.net = tf.keras.Sequential([
            tf.keras.layers.Conv2D(filters=96, kernel_size=11, strides=4, activation='relu'),
            tf.keras.layers.MaxPooling2D(pool_size=3, strides=2),
            tf.keras.layers.Conv2D(filters=256, kernel_size=5, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(pool_size=3, strides=2),
            tf.keras.layers.Conv2D(filters=384, kernel_size=3, activation='relu', padding='same'),
            tf.keras.layers.Conv2D(filters=384, kernel_size=3, activation='relu', padding='same'),
            tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(pool_size=3, strides=2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(4096, activation='relu'),
            tf.keras.layers.Dense(4096, activation='relu'),
            tf.keras.layers.Dense(num_classes)
        ])
    def call(self, x):
        return self.net(x)
        

In [8]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

def preprocess(image):
    image = tf.expand_dims(image, axis=-1)
    image = tf.image.resize(image, [224, 224]) #AlexNet takes a 224x224 image the fashion MNIST images are 28x28
    image = image.astype('float32')/255.0
    return image
x_train = preprocess(x_train)
x_test = preprocess(x_test)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))

batch_size= 128
train_ds = tf.data.Dataset.shuffle(buffer_size=1024).batch(batch_size)
test_de = tf.data.Dataset.batch(batch_size)

2025-10-04 16:19:32.765877: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (mklcpu) ran out of memory trying to allocate 11.21GiB (rounded to 12042240000)requested by op ResizeBilinear
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-10-04 16:19:32.766389: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1049] BFCAllocator dump for mklcpu
2025-10-04 16:19:32.766740: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1056] Bin (256): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2025-10-04 16:19:32.766750: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1056] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2025-10-04 16:19:32.766753: I extern

ResourceExhaustedError: {{function_node __wrapped__ResizeBilinear_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[60000,224,224,1] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:ResizeBilinear] name: 

In [24]:
model = AlexNet()
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.sgd(lr=0.01),
              metrics=['accuracy'])

TypeError: sparse_categorical_crossentropy() missing 2 required positional arguments: 'y_true' and 'y_pred'

In [22]:
x = tf.constant(tf.random.normal([28,28]))
x.shape
x = tf.expand_dims(x, axis=-1)
x = tf.image.resize(x, [224,224])
x.shape

TensorShape([224, 224, 1])